In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import os


C:\Users\Idriss\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
base_dir = r"C:\Users\Idriss\Desktop\PFA final sumbmission\archive(2)"
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')
IMG_SIZE = (150, 150)
BATCH_SIZE = 32
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)


Found 557 images belonging to 2 classes.
Found 140 images belonging to 2 classes.


In [3]:
# Load the VGG16 model without the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze the layers except the last 4 layers
for layer in base_model.layers[:-4]:
    layer.trainable = False

# Add custom layers on top of the base model
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [4]:
# Define the number of epochs
EPOCHS = 20

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=test_generator,
    validation_steps=test_generator.samples // BATCH_SIZE,
    epochs=EPOCHS
)


Epoch 1/20
17/17 [==============================] - 31s 2s/step - loss: 0.5833 - accuracy: 0.6686 - val_loss: 0.5272 - val_accuracy: 0.7500
Epoch 2/20
17/17 [==============================] - 29s 2s/step - loss: 0.3989 - accuracy: 0.8171 - val_loss: 0.4524 - val_accuracy: 0.7734
Epoch 3/20
17/17 [==============================] - 32s 2s/step - loss: 0.3133 - accuracy: 0.8610 - val_loss: 0.2931 - val_accuracy: 0.8438
Epoch 4/20
17/17 [==============================] - 32s 2s/step - loss: 0.2421 - accuracy: 0.8857 - val_loss: 0.4135 - val_accuracy: 0.8594
Epoch 5/20
17/17 [==============================] - 32s 2s/step - loss: 0.2092 - accuracy: 0.9200 - val_loss: 0.3374 - val_accuracy: 0.8281
Epoch 6/20
17/17 [==============================] - 32s 2s/step - loss: 0.1681 - accuracy: 0.9371 - val_loss: 0.3801 - val_accuracy: 0.7812
Epoch 7/20
17/17 [==============================] - 32s 2s/step - loss: 0.1266 - accuracy: 0.9543 - val_loss: 0.5511 - val_accuracy: 0.8359
Epoch 8/20
17/17 [==

In [5]:
# Evaluate the model on the validation set
loss, accuracy = model.evaluate(test_generator, steps=test_generator.samples // BATCH_SIZE)
print(f'Validation Loss: {loss}')
print(f'Validation Accuracy: {accuracy}')


4/4 [==============================] - 5s 980ms/step - loss: 0.8522 - accuracy: 0.8516
Validation Loss: 0.8521666526794434
Validation Accuracy: 0.8515625


In [6]:
# Save the model
model.save('dog_cat_classifier_vgg16.h5')


C:\Users\Idriss\anaconda3\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [2]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from tkinter import Tk, filedialog
import numpy as np

# Load the trained model
model = load_model('dog_cat_classifier_vgg16.h5')

def preprocess_image(img_path):
    # Load the image with target size
    img = image.load_img(img_path, target_size=(150, 150))
    # Convert the image to array
    img_array = image.img_to_array(img)
    # Expand dimensions to match the model's input shape
    img_array = np.expand_dims(img_array, axis=0)
    # Rescale the image
    img_array /= 255.0
    return img_array

def predict_image(model, img_path):
    # Preprocess the image
    processed_image = preprocess_image(img_path)
    # Make a prediction
    prediction = model.predict(processed_image)
    # Determine the class based on the prediction
    if prediction[0] > 0.5:
        return "Dog"
    else:
        return "Cat"

def choose_image():
    # Create a Tkinter root window (hidden)
    root = Tk()
    root.withdraw()  # Hide the root window
    # Open file dialog and return the selected file path
    file_path = filedialog.askopenfilename(title="Choose an image",
                                           filetypes=[("Image files", "*.jpg;*.jpeg;*.png")])
    return file_path

# Select an image
img_path = choose_image()

# Check if a file was selected
if img_path:
    # Predict the class
    result = predict_image(model, img_path)
    print(f'The image is a: {result}')
else:
    print("No image selected.")


1/1 [==============================] - 1s 796ms/step
The image is a: Dog
